GDAL提供了两个函数来访问影像的数值。
- ReadRaster() 读取图像数据(以二进制的形式)
- ReadAsArray() 读取图像数据(以数组的形式)

In [1]:
from osgeo import gdal
dataset = gdal.Open("/gdata/lu75c.tif")
help(dataset.ReadRaster)
help(dataset.ReadAsArray)

Help on method ReadRaster in module osgeo.gdal:

ReadRaster(xoff=0, yoff=0, xsize=None, ysize=None, buf_xsize=None, buf_ysize=None, buf_type=None, band_list=None, buf_pixel_space=None, buf_line_space=None, buf_band_space=None, resample_alg=0, callback=None, callback_data=None) method of osgeo.gdal.Dataset instance

Help on method ReadAsArray in module osgeo.gdal:

ReadAsArray(xoff=0, yoff=0, xsize=None, ysize=None, buf_obj=None, buf_xsize=None, buf_ysize=None, buf_type=None, resample_alg=0, callback=None, callback_data=None, interleave='band') method of osgeo.gdal.Dataset instance
    Reading a chunk of a GDAL band into a numpy array. The optional (buf_xsize,buf_ysize,buf_type)
    parameters should generally not be specified if buf_obj is specified. The array is returned



- xoff, yoff ：指定要读取部分的原点位置在整张图像中距离全图原点的位置(以像元为单位）。
- xsize, ysize ： 指定要读取部分图像的矩形的长和宽（以像元为单位）。
- buf_xsize, buf_ysize ：用于在读取出一部分图像后进行缩放。使用这两个参数来**定义缩放后图像最终的宽和高**。
- buf_type ：可以对读出数据的类型进行转换（比如原图数据类型是short，而想把它们缩小成byte）。
- band_list ：在多波段的情况下指定要读取的波段。
- 注意，ReadAsArray中的`buf_obj`意思是数组类型。如果指定了，则不用再输buf的其它参数了

## band.ReadAsArray()函数的几个参数的意义

In [2]:
from gdalconst import *
ds = gdal.Open("/gdata/geotiff_file.tif")
band = ds.GetRasterBand(1)
band.ReadAsArray(100,100,5,5,10,10)

array([[236, 236, 237, 237, 237, 237, 237, 237, 227, 227],
       [236, 236, 237, 237, 237, 237, 237, 237, 227, 227],
       [235, 235, 232, 232, 233, 233, 234, 234, 225, 225],
       [235, 235, 232, 232, 233, 233, 234, 234, 225, 225],
       [242, 242, 235, 235, 232, 232, 233, 233, 224, 224],
       [242, 242, 235, 235, 232, 232, 233, 233, 224, 224],
       [254, 254, 244, 244, 238, 238, 237, 237, 229, 229],
       [254, 254, 244, 244, 238, 238, 237, 237, 229, 229],
       [246, 246, 246, 246, 248, 248, 250, 250, 235, 235],
       [246, 246, 246, 246, 248, 248, 250, 250, 235, 235]], dtype=uint8)

前两个100（`xoff`和`yoff`）是取值窗口的左上角在实际数据中所处象元的(x, y)位置，中间两个5（`xsize`和`ysize`）是取值窗口覆盖的区域大小，后面两个10（`buf_xsize`, `buf_ysize`）是取值窗口取出数组进行缩放后数组的大小。

需要注意的是这里的buffer大小是根据参数自动分配的，可以不指定，若不指定则和第3，4个参数一致。5，6两个参数设置了缩放。假如取值窗口大小是20 × 20，取出后数组就可以人工设置大小。 让它成为10 × 10的数组，或者是40 × 40的数组。 如果设置成20 × 40的数组则取出的数组对于真实图像来说有了变形。

这个作用就是**把指定区域（前四个参数定义）的数据按比例缩放到用户指定区域（后两个定义）内**，必要的时候进行缩放。

## 获取数据的尺寸

上面是对band进行`ReadAsArray`处理的，实际上对ds也可以，他们的差别是：
- 对ds进行转换，转换结果是一个三维数组，第一个维度是波段数；
- 对band进行转换，转换的结果直接是一个二维数组

### ds.ReadAsArray().shape 从数据集直接获取

In [8]:
print(ds.ReadAsArray().shape)

(3, 900, 1500)


### 运用循环依次从波段上获取

In [9]:
for b in range(ds.RasterCount):
    # GDAL中的band计数是从1开始的
    band = ds.GetRasterBand(b + 1)
    band = band.ReadAsArray()
    print(f'波段大小：{band.shape}')

波段大小：(900, 1500)
波段大小：(900, 1500)
波段大小：(900, 1500)
